## Assignment 6

## We have not run the codes because when running them the size exceeds the allowed limit (from 100 maximum, we obtain more than 350 mb). 

In [ ]:
#We add the folium library
!pip install folium
!pip install geopandas

In [ ]:

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [ ]:
import matplotlib.pyplot as plt 
import folium 
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
# Get data of labor
sex_work = pd.read_stata( r'/Users/pabloparedeschavez/Documents/GitHub/MediaLab_Summer_Python/_data/8_trab_sex_20.dta' )
sex_work[ 'dpt_code' ] = sex_work.cod_ubigeo.str[ :2 ].copy()
sex_work[ 'prov_code' ] = sex_work.cod_ubigeo.str[ :4 ].copy()

# Sex work
women_work = sex_work[ sex_work.sex == 'Mujer' ].copy().reset_index( drop = True )

In [ ]:
# get data from lima
women_work[ 'month' ] = pd.to_datetime( women_work.month , format = '%B' ) \
                                    .dt.strftime( '%m' ) \
                                    .astype( int )

In [ ]:
# Sort by department and month
women_work.sort_values([ 'dpt_code', 'month'], inplace = True )

# Get the total number of women workers by dpt
dpt_women_work = women_work.groupby( [ 'dpt_code', 'month'], as_index = False )[['empl']] \
                            .sum() \
                            .rename( columns = {'empl' :'women_empl'})

# Sort by dpt code and month
dpt_women_work.sort_values([ 'dpt_code', 'month'], inplace = True )

In [ ]:
df2 = dpt_women_work.groupby( ['dpt_code'],as_index = False )[['women_empl']].mean()


# Shapefiles
# Map 1
We replicate the map  using Folium also we added the GeoJSON data

In [ ]:
dpt_shp = gpd.read_file( r'/Users/pabloparedeschavez/Documents/GitHub/MediaLab_Summer_Python/_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp' )

In [ ]:
print(dpt_shp.columns)
#Just to check the list to organize the information

In [ ]:
print(dpt_shp.head())
##Just to check the full list to organize the information

In [ ]:
df3 = dpt_shp.merge( df2, left_on = 'CCDD', right_on = 'dpt_code'  )

In [ ]:
import branca.colormap as cm  
import json  
 
df3_json = df3.to_json()

# Creating the map
m = folium.Map(location=[-9.1900, -75.0152], zoom_start=6)   

# Colormap is needed
colormap = cm.linear.Reds_09.scale(df3['women_empl'].min(), df3['women_empl'].max())

 
folium.GeoJson(
    df3_json,
    name='Women Employment',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['women_empl']),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

# Colormap for the map
colormap.add_to(m)
 
m

 

# Map 2

Again we added the GeoJSON data

In [ ]:
m = folium.Map(location=[-9.1900, -75.0152], zoom_start=6)  


bins = [20000, 40000, 60000, 100000]

# Colormap will be needed
colormap = cm.LinearColormap(colors=['#fee5d9','#fcae91','#fb6a4a','#de2d26'], index=bins, vmin=df3['women_empl'].min(), vmax=df3['women_empl'].max())


folium.GeoJson(
    df3_json,
    name='Women Employment',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['women_empl']) if feature['properties']['women_empl'] is not None else 'transparent',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

# Add the colormap to the map
colormap.caption = 'Number of Employers'
colormap.add_to(m)

 
m

# Map 3
We skip to show Lima

In [ ]:
df2_=df2[df2.dpt_code!='15']
df3_=df3[df3.CCDD!='15']

In [ ]:
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z = folium.Map(location = [lat_palacio, long_palacio], tiles='OpenStreetMap', zoom_start = zoom_start)

bins=[x*10000 for x in range(8)] 

folium.Choropleth(
    geo_data=df3_, 
    data=df2_, 
    columns=['dpt_code', 'women_empl'], 
    key_on="feature.properties.dpt_code", 
    fill_color="Reds", 
    line_color = "#0a0a0a", 
    line_dash_array = '5, 5',
    threshold_scale=bins, 
    ).add_to(z)

z

# Map4

In [ ]:
# This is the original code of the assignment:
df4 = sex_work.groupby( ['dpt_code', 'month', 'sex'], as_index = False )[['empl']].sum() \
        .pivot( index = [ 'dpt_code', 'month' ] , columns = 'sex',values='empl') \
        .reset_index()
df4

In [ ]:
# This is the original code of the assignment:
df4[ 'prop_wom' ] = ( df4.Mujer * 100 / df4.Hombre )
 


In [ ]:
# This is the original code of the assignment:
df5 = dpt_shp.merge( df4, left_on = 'CCDD', right_on = 'dpt_code'  )


In [ ]:
# This is the original code of the assignment:
df4 = sex_work.groupby( ['dpt_code', 'month', 'sex'], as_index = False )[['empl']].sum() \
        .pivot( index = [ 'dpt_code', 'month' ] , columns = 'sex',values='empl') \
        .reset_index()

 
df4[ 'prop_wom' ] = ( df4.Mujer * 100 / df4.Hombre )

df5 = dpt_shp.merge( df4, left_on = 'CCDD', right_on = 'dpt_code'  )

fig, axis = plt.subplots( nrows = 4, ncols= 3, figsize = ( 15, 15 ) )


idx = 0
for i in range( 4 ):
    for j in range ( 3 ):
        
        
        ax = axis[ i ][ j ]
        
        month = df5.month.unique()[ idx ]
        
        df6 = df5[ df5.month == month ]
        
        df6.plot( column='prop_wom', 
                  cmap='Reds', 
                  linestyle='--',
                  edgecolor='black', 
                  legend = True, 
                  ax = ax 
                )
        
        ax.set_title( month )
        
        idx = idx + 1

In [ ]:
#We add new breakpoints to the bins and  we added the choropleth to the map. This is the example for December. 

In [ ]:
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z = folium.Map(location = [lat_palacio, long_palacio], tiles='OpenStreetMap', zoom_start = zoom_start)

bins=[x*10 for x in range(1,9)]  

for i in range(12):
    
     
    if i==0:
        globals()[f'd_{i}']=folium.Choropleth(    
        geo_data=df6,  
        data=df6, 
        columns=['dpt_code', 'prop_wom'],  
        key_on="feature.properties.dpt_code", 
        fill_color="Reds", 
        line_color = "#0a0a0a", 
        line_dash_array = '5, 5',
        threshold_scale=bins, 
        control=True,  
        name=month 
        )
    
    else:
        globals()[f'd_{i}']=folium.Choropleth(
        geo_data=df6, 
        data=df6, 
        columns=['dpt_code', 'prop_wom'], 
        key_on="feature.properties.dpt_code", 
        fill_color="Reds", 
        line_color = "#0a0a0a", 
        line_dash_array = '5, 5',
        threshold_scale=bins,
        control=True,
        name=month,
        show=False   
        )
        
          
        for key in globals()[f'd_{i}']._children:   
            if key.startswith('color_map'):        
                del(globals()[f'd_{i}']._children[key]) # 
         
    
    globals()[f'd_{i}'].add_to(z)
    
folium.LayerControl(collapsed=False).add_to(z)   

z

In [ ]:
#This is other way to get the maps by months (second option).


import folium

lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z = folium.Map(location=[lat_palacio, long_palacio], tiles='OpenStreetMap', zoom_start=zoom_start)

bins = [x * 10 for x in range(1, 9)]

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

for i, month in enumerate(months):
    if i == 0:
        globals()[f'd_{i}'] = folium.Choropleth(
            geo_data=df6,
            data=df6,
            columns=['dpt_code', 'prop_wom'],
            key_on="feature.properties.dpt_code",
            fill_color="Reds",
            line_color="#0a0a0a",
            line_dash_array='5, 5',
            threshold_scale=bins,
            control=True,
            name=month
        )
    else:
        globals()[f'd_{i}'] = folium.Choropleth(
            geo_data=df6,
            data=df6,
            columns=['dpt_code', 'prop_wom'],
            key_on="feature.properties.dpt_code",
            fill_color="Reds",
            line_color="#0a0a0a",
            line_dash_array='5, 5',
            threshold_scale=bins,
            control=True,
            name=month,
            show=False
        )

        for key in globals()[f'd_{i}']._children:
            if key.startswith('color_map'):
                del (globals()[f'd_{i}']._children[key])

    globals()[f'd_{i}'].add_to(z)

folium.LayerControl(collapsed=False).add_to(z)

z


## Map 5

In [ ]:
# We include the column "prop_woman" in the DataFrame
df6[ 'prop_wom' ] = ( df6.Mujer * 100 / df6.Hombre )
df7 = dpt_shp.merge( df6, left_on = 'CCDD', right_on = 'dpt_code'  )
df7.head()

In [ ]:
# Delete all rows that have 'LIMA' in the 'NOMBDEP' column
df8 = df7[df7['NOMBDEP_x'] != 'LIMA']

# Ensure indexes are consecutive after deletion
df8.reset_index(drop=True, inplace=True)

# Show the new DataFrame df8
df8.head()

In [ ]:
import folium
import geopandas as gpd
from folium.plugins import HeatMap

# Copy the DataFrame to avoid SettingWithCopyWarning
df8_copy = df8.copy()

# Delete all rows that have 'LIMA' in the 'NOMBDEP' column
df8_copy = df8_copy[df8_copy['NOMBDEP_x'] != 'LIMA']

# Ensure indexes are consecutive after deletion
df8_copy.reset_index(drop=True, inplace=True)

# The coordinates of the government palace will be used as the centroid
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

# Create a map object
m = folium.Map(location=[lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start=zoom_start)

# Create the Choropleth and add it to the map
choropleth = folium.Choropleth(
    geo_data=df8_copy,
    data=df8_copy,
    columns=['dpt_code', 'prop_wom'],
    key_on="feature.properties.dpt_code",
    fill_color="Reds",
    fill_opacity=0.8,
    legend_name="Women Proportion",
    bins=[17.01, 20, 30, 40, 50, 100]
).add_to(m)

# Show the map
m
